# 실전 문제 해결 (과소적합)

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 10

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, use_bias=False)
        self.batch1 = tf.keras.layers.BatchNormalization()
        
        self.dense2 = tf.keras.layers.Dense(32, use_bias=False)
        self.batch2 = tf.keras.layers.BatchNormalization()
         
        self.dense3 = tf.keras.layers.Dense(64, use_bias=False)
        self.batch3 = tf.keras.layers.BatchNormalization()

        self.dense4 = tf.keras.layers.Dense(128, use_bias=False)
        self.batch4 = tf.keras.layers.BatchNormalization()
        
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.batch1(x, training)
        x = tf.nn.relu(x)
        
        h = self.batch2(x, training)
        h = tf.nn.relu(h)
        h = self.dense2(h)
        x = tf.concat([x, h],axis=-1)
        
        h = self.batch3(x, training)
        h = tf.nn.relu(h)
        h = self.dense3(h)
        x = tf.concat([x, h],axis=-1)

        h = self.batch4(x, training)
        h = tf.nn.relu(h)
        h = self.dense4(h)
        x = tf.concat([x, h],axis=-1)
        
        return self.dense5(x)

## 데이터셋 준비


In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

4423680/4422102 [==============================] - 0s 0us/step


## Keras API 모델 학습

In [5]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5240 - accuracy: 0.8133 - val_loss: 0.5448 - val_accuracy: 0.8064
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4054 - accuracy: 0.8526 - val_loss: 0.3905 - val_accuracy: 0.8599
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3682 - accuracy: 0.8661 - val_loss: 0.3957 - val_accuracy: 0.8561
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3494 - accuracy: 0.8723 - val_loss: 0.3942 - val_accuracy: 0.8587
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3328 - accuracy: 0.8789 - val_loss: 0.4151 - val_accuracy: 0.8532
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3184 - accuracy: 0.8830 - val_loss: 0.3650 - val_accuracy: 0.8700
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3084 - accuracy: 0.8877 - val_loss: 0.3726 - val_accuracy: